In [164]:
import json
import sys
from base64 import b64encode
from base64 import b64decode
from Crypto.Cipher import AES
from Crypto.Cipher import ChaCha20_Poly1305
from Crypto.Random import get_random_bytes
from time import process_time
import tracemalloc

In [165]:
for iter in range(4):
    print("\n\nArquivo "+str(iter))
    # Dados
    path = "dados/texto_"+str(iter+1)+".txt"
    with open (path, "r") as arquivoTexto:
        texto = arquivoTexto.read()
    print("Tamanho do arquivo txt: "+str(round(sys.getsizeof(texto)*0.001,2))+"KB")
    # AES - encryption - Cifra de Bloco simétrica - CCM mode
    print("\tAES Criptografando")
    tracemalloc.start()
    aes_encrypt_time = process_time()
    header = b"header"

    # Carregando dados para criptografar
    data = texto.encode('UTF-8')
    # Chave 16-bytes AES-128
    key = get_random_bytes(16)

    # Inicializacao
    tracemalloc.clear_traces()
    cipher = AES.new(key, AES.MODE_CCM)
    init_encrypt_aes_memory, init_encrypt_aes_memory_peak = tracemalloc.get_traced_memory()

    # Hashing
    tracemalloc.clear_traces()
    cipher.update(header)
    hashing_encrypt_aes_memory, hashing_encrypt_aes_memory_peak = tracemalloc.get_traced_memory()

    # Criptografia
    tracemalloc.clear_traces()
    ciphertext, tag = cipher.encrypt_and_digest(data)
    encrypt_aes_memory, encrypt_aes_memory_peak = tracemalloc.get_traced_memory()

    # Calculando Tempo
    aes_encrypt_elapsed_time = process_time() - aes_encrypt_time

    # Montando arquivo JSON com dados da criptografia
    json_k = [ 'nonce', 'header', 'ciphertext', 'tag' ]
    json_v = [ b64encode(x).decode('UTF-8') for x in (cipher.nonce, header, ciphertext, tag) ]
    result = json.dumps(dict(zip(json_k, json_v)))

    # Calculando estatisticas
    init_encrypt_aes_memory = init_encrypt_aes_memory/10**3
    init_encrypt_aes_memory_peak = (init_encrypt_aes_memory_peak/10**3)#0,000010
    hashing_encrypt_aes_memory = hashing_encrypt_aes_memory/10**3
    hashing_encrypt_aes_memory_peak = (hashing_encrypt_aes_memory_peak/10**3)
    encrypt_aes_memory = encrypt_aes_memory/10**3
    encrypt_aes_memory_peak = (encrypt_aes_memory_peak/10**3)

    # Mostrando Resultados
    #print(result)
    print("\t\tUso de memoria na inicializacao: ", init_encrypt_aes_memory, "KB. Pico: ", init_encrypt_aes_memory_peak, "KB")
    print("\t\tUso de memoria no hashing: ", hashing_encrypt_aes_memory, "KB. Pico: ", hashing_encrypt_aes_memory, "KB")
    print("\t\tUso de memoria para criptografar: ", encrypt_aes_memory, "KB. Pico: ", encrypt_aes_memory_peak, "KB")
    print("\t\tAES encryption time: "+str(aes_encrypt_elapsed_time)+"s")
    print("\tAES encryption - Complete\n")

    # AES - decryption
    print("\tAES Descriptografando")
    try:
        aes_decrypt_time = process_time()

        # Montando mensagem para descriptografar
        b64 = json.loads(result)
        jv = {k:b64decode(b64[k]) for k in json_k}

        # Inicialiazacao
        tracemalloc.clear_traces()
        cipher = AES.new(key, AES.MODE_CCM, nonce=jv['nonce'])
        init_decrypt_aes_memory, init_decrypt_aes_memory_peak = tracemalloc.get_traced_memory()

        # Hashing
        tracemalloc.clear_traces()
        cipher.update(jv['header'])
        hashing_decrypt_aes_memory, hashing_decrypt_aes_memory_peak = tracemalloc.get_traced_memory()

        # Descriptografia
        tracemalloc.clear_traces()
        plaintext = cipher.decrypt_and_verify(jv['ciphertext'], jv['tag'])
        decrypt_aes_memory, decrypt_aes_memory_peak = tracemalloc.get_traced_memory()

        # Calculando Estatisticas
        aes_decrypt_elapsed_time = process_time() - aes_decrypt_time
        init_decrypt_aes_memory = init_decrypt_aes_memory/10**3
        init_decrypt_aes_memory_peak = (init_decrypt_aes_memory_peak/10**3)
        hashing_decrypt_aes_memory = hashing_decrypt_aes_memory/10**3
        hashing_decrypt_aes_memory_peak = (hashing_decrypt_aes_memory_peak/10**3)
        decrypt_aes_memory = decrypt_aes_memory/10**3
        decrypt_aes_memory_peak = (decrypt_aes_memory_peak/10**3)

        # Mostrando Resultados
        #print("The message was: " + format(plaintext.decode("UTF-8")))
        print("\t\tUso de memoria na inicializacao: ", init_decrypt_aes_memory, "KB. Pico: ", init_decrypt_aes_memory_peak, "KB")
        print("\t\tUso de memoria no hashing: ", hashing_decrypt_aes_memory, "KB. Pico: ", hashing_decrypt_aes_memory, "KB")
        print("\t\tUso de memoria para descriptografar: ", decrypt_aes_memory, "KB. Pico: ", decrypt_aes_memory_peak, "KB")
        print("\t\tAES decryption time: "+str(aes_decrypt_elapsed_time)+"s")
        print("\tAES decryption - Complete")
    except (ValueError, KeyError):
        print("\tIncorrect decryption")

    # ChaCha20-Poly1305 - encryption - Cifra de Fluxo - com funcao MAC
    print("\n\tChaCha20-Poly1305 Criptografando")
    chacha_encrypt_time = process_time()
    header = b"header"

    # Carregando dados para criptografar
    plaintext = texto.encode('UTF-8')

    # Chave de 32 bytes
    key = get_random_bytes(32)

    # Inicializacao
    tracemalloc.clear_traces()
    cipher = ChaCha20_Poly1305.new(key=key)
    init_encrypt_chacha_memory, init_encrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

    # Protecao para os dados associados
    tracemalloc.clear_traces()
    cipher.update(header)
    hashing_encrypt_chacha_memory, hashing_encrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

    # Criptografia
    tracemalloc.clear_traces()
    ciphertext, tag = cipher.encrypt_and_digest(plaintext)
    encrypt_chacha_memory, encrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

    # Calculando Tempo
    chacha_encrypt_elapsed_time = process_time() - chacha_encrypt_time

    # Montando arquivo JSON com os dados
    jk = [ 'nonce', 'header', 'ciphertext', 'tag' ]
    jv = [ b64encode(x).decode('utf-8') for x in (cipher.nonce, header, ciphertext, tag) ]
    result = json.dumps(dict(zip(jk, jv)))

    # Calculando Estatisticas
    init_encrypt_chacha_memory = init_encrypt_chacha_memory/10**3
    init_encrypt_chacha_memory_peak = (init_encrypt_chacha_memory_peak/10**3)
    hashing_encrypt_chacha_memory = hashing_encrypt_chacha_memory/10**3
    hashing_encrypt_chacha_memory_peak = (hashing_encrypt_chacha_memory_peak/10**3)
    encrypt_chacha_memory = encrypt_chacha_memory/10**3
    encrypt_chacha_memory_peak = (encrypt_chacha_memory_peak/10**3)

    # Mostrando Resultados
    #print(result)
    print("\t\tUso de memoria na inicializacao: ", init_encrypt_chacha_memory, "KB. Pico: ", init_encrypt_chacha_memory_peak, "KB")
    print("\t\tUso de memoria no hashing: ", hashing_encrypt_chacha_memory, "KB. Pico: ", hashing_encrypt_chacha_memory_peak, "KB")
    print("\t\tUso de memoria para criptografar: ", encrypt_chacha_memory, "KB. Pico: ", encrypt_chacha_memory_peak, "KB")
    print("\t\tChaCha20-Poly1305 encryption time: "+str(chacha_encrypt_elapsed_time)+"s")
    print("\tChaCha20-Poly1305 encryption - Complete\n")

    # ChaCha20-Poly1305 - decryption
    print("\tChaCha20-Poly1305 Descriptografando")
    try:
        chacha_decrypt_time = process_time()

        # Montando mensagem para descriptografar
        b64 = json.loads(result)
        jk = [ 'nonce', 'header', 'ciphertext', 'tag' ]
        jv = {k:b64decode(b64[k]) for k in jk}

        # Inicialiazacao
        tracemalloc.clear_traces()
        cipher = ChaCha20_Poly1305.new(key=key, nonce=jv['nonce'])
        init_decrypt_chacha_memory, init_decrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

        # Protecao para os dados associados
        tracemalloc.clear_traces()
        cipher.update(jv['header'])
        hashing_decrypt_chacha_memory, hashing_decrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

        # Descriptografia
        tracemalloc.clear_traces()
        plaintext = cipher.decrypt_and_verify(jv['ciphertext'], jv['tag'])
        decrypt_chacha_memory, decrypt_chacha_memory_peak = tracemalloc.get_traced_memory()

        # Calculando Estatisticas
        chacha_decrypt_elapsed_time = process_time() - chacha_decrypt_time
        init_decrypt_chacha_memory = init_decrypt_chacha_memory/10**3
        init_decrypt_chacha_memory_peak = (init_decrypt_chacha_memory_peak/10**3)
        hashing_decrypt_chacha_memory = hashing_decrypt_chacha_memory/10**3
        hashing_decrypt_chacha_memory_peak = (hashing_decrypt_chacha_memory_peak/10**3)
        decrypt_chacha_memory = decrypt_chacha_memory/10**3
        decrypt_chacha_memory_peak = (decrypt_chacha_memory_peak/10**3)

        # Mostrando Resultados
        #print("The message was: " + format(plaintext.decode("UTF-8")))
        print("\t\tUso de memoria na inicializacao: ", init_decrypt_chacha_memory, "KB. Pico: ", init_decrypt_chacha_memory_peak, "KB")
        print("\t\tUso de memoria no hashing: ", hashing_decrypt_chacha_memory, "KB. Pico: ", hashing_decrypt_chacha_memory, "KB")
        print("\t\tUso de memoria para descriptografar: ", decrypt_chacha_memory, "KB. Pico: ", decrypt_chacha_memory_peak, "KB")
        print("\t\tChaCha20-Poly1305 decryption time: "+str(chacha_decrypt_elapsed_time)+"s")
        print("\tChaCha20-Poly1305 decryption - Complete\n")
    except (ValueError, KeyError):
        print("\tIncorrect decryption")



Arquivo 0
Tamanho do arquivo txt: 6344.49KB
	AES Criptografando
		Uso de memoria na inicializacao:  1.106 KB. Pico:  2.759 KB
		Uso de memoria no hashing:  0.072 KB. Pico:  0.072 KB
		Uso de memoria para criptografar:  6349.539 KB. Pico:  12694.072 KB
		AES encryption time: 0.015625s
	AES encryption - Complete

	AES Descriptografando
		Uso de memoria na inicializacao:  1.062 KB. Pico:  2.731 KB
		Uso de memoria no hashing:  0.072 KB. Pico:  0.072 KB
		Uso de memoria para descriptografar:  6344.73 KB. Pico:  19033.74 KB
		AES decryption time: 0.046875s
	AES decryption - Complete

	ChaCha20-Poly1305 Criptografando
		Uso de memoria na inicializacao:  0.821 KB. Pico:  0.909 KB
		Uso de memoria no hashing:  0.0 KB. Pico:  0.056 KB
		Uso de memoria para criptografar:  6350.078 KB. Pico:  6350.146 KB
		ChaCha20-Poly1305 encryption time: 0.03125s
	ChaCha20-Poly1305 encryption - Complete

	ChaCha20-Poly1305 Descriptografando
		Uso de memoria na inicializacao:  0.908 KB. Pico:  1.036 KB
		Uso 